In [3]:
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon
from pathlib import Path

# --- Terminal Polygon for Bergen ---
terminal_poly = Polygon([
    (5.3128688,60.3884717),(5.3080408,60.3872418),
    (5.3123326,60.3854181),(5.3159372,60.383838),
    (5.317643,60.3840554),(5.3189305,60.3850946),
    (5.3141137,60.387984),(5.3128688,60.3884717)
])

# --- Read Data ---
df = pd.read_csv('Bergen.csv', dayfirst=True, low_memory=False)
df['date_time_utc'] = pd.to_datetime(df['date_time_utc'], utc=True, errors='coerce')
df = df.dropna(subset=['date_time_utc'])

# --- Add Terminal Polygon Inclusion Boolean ---
df['in_terminal'] = df.apply(lambda row: terminal_poly.contains(Point(row['longitude'], row['latitude'])), axis=1)

# --- Sort data for processing ---
df = df.sort_values(['mmsi', 'date_time_utc']).reset_index(drop=True)

summary_rows = []

# --- Process per Vessel ---
for mmsi, vessel_df in df.groupby('mmsi'):
    vessel_df = vessel_df.sort_values('date_time_utc').reset_index(drop=True)
    vessel_name = vessel_df['ship_name'].iloc[0]
    vessel_type = vessel_df['ship_type'].iloc[0]
    vessel_length = vessel_df['length'].iloc[0]
    prev_time = None
    visit_num = 0
    visit_rows = []
    visit_start_idx = 0

    # Identify visits: Split where gap >10 hours
    vessel_df['prev_time'] = vessel_df['date_time_utc'].shift(1)
    vessel_df['hour_gap'] = (vessel_df['date_time_utc'] - vessel_df['prev_time']).dt.total_seconds()/3600
    vessel_df.loc[vessel_df['hour_gap'].isnull(), 'hour_gap'] = 0
    visit_breaks = vessel_df.index[vessel_df['hour_gap'] > 10].tolist()
    visit_breaks = [0] + visit_breaks + [len(vessel_df)]
    
    # For each visit
    for i in range(len(visit_breaks)-1):
        visit_num += 1
        visit_start = visit_breaks[i]
        visit_end = visit_breaks[i+1]
        visit_df = vessel_df.iloc[visit_start:visit_end].copy()
        visit_start_time = visit_df['date_time_utc'].iloc[0]
        visit_end_time = visit_df['date_time_utc'].iloc[-1]
        visit_duration_hr = (visit_end_time - visit_start_time).total_seconds()/3600
        
        # Stops in port (speed<1), with breakdown
        visit_df['stopped'] = visit_df['speed_over_ground'] < 1
        visit_df['stopped_shift'] = visit_df['stopped'].shift(1, fill_value=False)
        stop_starts = visit_df.index[(visit_df['stopped']) & (~visit_df['stopped_shift'])].tolist()
        stop_ends = visit_df.index[(~visit_df['stopped']) & (visit_df['stopped_shift'])].tolist()
        # Handle if still stopped at end
        if visit_df['stopped'].iloc[-1]:
            stop_ends.append(visit_df.index[-1])
        # Ignore if unmatched stop start
        if len(stop_ends) and (not len(stop_starts) or stop_ends[0] < stop_starts[0]):
            stop_starts = [visit_df.index[0]] + stop_starts
        stops = []
        stop_durations_total = 0
        stop_durations_terminal = 0
        stop_count = 0
        stop_count_terminal = 0
        for s, e in zip(stop_starts, stop_ends):
            stop_start_time = visit_df.loc[s, 'date_time_utc']
            stop_end_time = visit_df.loc[e, 'date_time_utc']
            stop_duration = (stop_end_time - stop_start_time).total_seconds()/3600
            in_terminal = visit_df.loc[s:e, 'in_terminal'].any()
            stops.append({
                "stop_start": stop_start_time,
                "stop_end": stop_end_time,
                "duration_hr": stop_duration,
                "in_terminal": in_terminal
            })
            stop_count += 1
            stop_durations_total += stop_duration
            if in_terminal:
                stop_count_terminal += 1
                stop_durations_terminal += stop_duration

        # Time spent in terminal polygon for this visit
        terminal_times = visit_df[visit_df['in_terminal']]
        nonterminal_times = visit_df[~visit_df['in_terminal']]
        if not terminal_times.empty:
            terminal_duration = (terminal_times['date_time_utc'].iloc[-1] - terminal_times['date_time_utc'].iloc[0]).total_seconds()/3600
        else:
            terminal_duration = 0
        nonterminal_duration = visit_duration_hr - terminal_duration

        # Time since last visit
        if i > 0:
            last_end = vessel_df.loc[visit_breaks[i]-1, 'date_time_utc']
            time_since_last_visit_hr = (visit_start_time - last_end).total_seconds()/3600
        else:
            time_since_last_visit_hr = np.nan

        # Compose summary row
        summary_rows.append({
            "mmsi": mmsi,
            "ship_name": vessel_name,
            "ship_type": vessel_type,
            "length": vessel_length,
            "visit_num": visit_num,
            "visit_start_time": visit_start_time,
            "visit_end_time": visit_end_time,
            "visit_duration_hr": round(visit_duration_hr, 2),
            "stops_count": stop_count,
            "stops_total_duration_hr": round(stop_durations_total, 2),
            "stops_terminal_count": stop_count_terminal,
            "stops_terminal_duration_hr": round(stop_durations_terminal, 2),
            "terminal_duration_hr": round(terminal_duration, 2),
            "nonterminal_duration_hr": round(nonterminal_duration, 2),
            "time_since_last_visit_hr": round(time_since_last_visit_hr, 2) if not np.isnan(time_since_last_visit_hr) else "",
            "stops_detail": str(stops)
        })

summary_df = pd.DataFrame(summary_rows)

# --- Save to CSV ---
summary_df.to_csv("Bergen_kpi_summary.csv", index=False)

print("Summary saved as Bergen_kpi_summary.csv")
display(summary_df.head(10))


Summary saved as Bergen_kpi_summary.csv


,mmsi,ship_name,ship_type,length,visit_num,visit_start_time,visit_end_time,visit_duration_hr,stops_count,stops_total_duration_hr,stops_terminal_count,stops_terminal_duration_hr,terminal_duration_hr,nonterminal_duration_hr,time_since_last_visit_hr,stops_detail
0,209762000,NCL SALTEN,70.0,135.0,1,2024-01-26 19:34:28+00:00,2024-01-26 21:30:50+00:00,1.94,1,1.51,1,1.51,1.42,0.52,,[{'stop_start': Timestamp('2024-01-26 19:49:28...
1,209762000,NCL SALTEN,70.0,135.0,2,2024-02-22 19:52:06+00:00,2024-02-22 21:19:22+00:00,1.45,1,1.25,1,1.25,1.25,0.20,646.35,[{'stop_start': Timestamp('2024-02-22 19:52:06...
2,209762000,NCL SALTEN,70.0,135.0,3,2024-02-29 06:31:59+00:00,2024-02-29 12:27:25+00:00,5.92,2,5.47,0,0.00,0.00,5.92,153.21,[{'stop_start': Timestamp('2024-02-29 06:47:03...
3,209762000,NCL SALTEN,70.0,135.0,4,2024-03-15 01:29:24+00:00,2024-03-15 03:46:12+00:00,2.28,2,1.82,1,1.77,1.88,0.40,349.03,[{'stop_start': Timestamp('2024-03-15 01:47:43...
4,209762000,NCL SALTEN,70.0,135.0,5,2024-03-29 20:25:59+00:00,2024-03-30 00:32:38+00:00,4.11,1,3.66,1,3.66,3.71,0.40,352.66,[{'stop_start': Timestamp('2024-03-29 20:40:59...
5,209762000,NCL SALTEN,70.0,135.0,6,2024-04-12 15:00:26+00:00,2024-04-12 16:22:54+00:00,1.37,1,0.97,1,0.97,1.02,0.36,326.46,[{'stop_start': Timestamp('2024-04-12 15:15:28...
6,209903000,CEMCOMMANDER,70.0,113.0,1,2024-04-09 00:42:27+00:00,2024-04-09 08:22:21+00:00,7.67,1,7.26,0,0.00,0.00,7.67,,[{'stop_start': Timestamp('2024-04-09 01:00:27...
7,209903000,CEMCOMMANDER,70.0,113.0,2,2024-04-23 13:12:11+00:00,2024-04-23 21:35:23+00:00,8.39,1,7.95,0,0.00,0.00,8.39,340.83,[{'stop_start': Timestamp('2024-04-23 13:27:11...
8,210515000,SOLVIK,70.0,90.0,1,2024-01-11 13:17:21+00:00,2024-01-12 00:15:38+00:00,10.97,2,10.57,0,0.00,0.00,10.97,,[{'stop_start': Timestamp('2024-01-11 13:32:32...
9,211553000,BENEDIKT,70.0,147.0,1,2024-01-06 01:21:25+00:00,2024-01-06 13:55:55+00:00,12.57,1,12.12,1,12.12,12.17,0.40,,[{'stop_start': Timestamp('2024-01-06 01:39:43...


In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc

# --- Load your summary, process for dashboard ---
summary = pd.read_csv("Bergen_kpi_summary.csv", parse_dates=['visit_start_time', 'visit_end_time'])

# FILTER: Exclude vessels where ANY visit has stops_count > 5
vessels_to_exclude = summary.loc[summary['stops_count'] > 5, 'mmsi'].unique()
summary = summary[~summary['mmsi'].isin(vessels_to_exclude)].reset_index(drop=True)

# Month column (using dt.month for grouping; avoids Period warning)
summary['month'] = summary['visit_start_time'].dt.strftime('%Y-%m')

# KPI Stats
n_vessels = summary['mmsi'].nunique()
n_vessels_terminal = summary.groupby('mmsi')['terminal_duration_hr'].sum().gt(0).sum()

# --- Prepare figures ---
monthly_counts = summary.groupby('month')['mmsi'].nunique().reset_index(name='unique_vessels')
visits_per_vessel = summary.groupby('mmsi')['visit_num'].max().reset_index(name='total_visits')
visits_hist = visits_per_vessel['total_visits'].value_counts().sort_index()
lengths = summary.groupby('mmsi')[['length', 'ship_type']].first().reset_index()
lengths = pd.merge(lengths, visits_per_vessel, on='mmsi')
visit_heatmap = summary.groupby(['month', 'ship_type'])['mmsi'].nunique().reset_index()
heatmap_pivot = visit_heatmap.pivot(index='month', columns='ship_type', values='mmsi').fillna(0)

# Plotly Express/Graph objects (all interactive!)
fig_monthly = px.bar(monthly_counts, x='month', y='unique_vessels', title='Unique Vessels Each Month')
fig_visits_hist = px.bar(x=visits_hist.index, y=visits_hist.values, title='Number of Port Visits per Vessel',
                         labels={'x':'Visits','y':'Vessel Count'})
fig_visits_pie = px.pie(values=visits_hist.values, names=visits_hist.index, hole=0.4, title='Vessels by Number of Port Visits')
fig_box_visit = px.box(summary, y='visit_duration_hr', title='Distribution of Visit Durations (hr)')
fig_box_term = px.box(summary, y='terminal_duration_hr', title='Time in Terminal per Visit (hr)')
fig_stops_hist = px.histogram(summary, x='stops_count', nbins=summary['stops_count'].max()+1, title='Stops Per Visit Distribution')
fig_scatter = px.scatter(lengths, x='length', y='total_visits', color='ship_type',
                         title="Ship Length vs Total Port Visits", labels={'length':'Length (m)', 'total_visits':'Total Visits'})
fig_heatmap = go.Figure(data=go.Heatmap(
    z=heatmap_pivot.values,
    x=heatmap_pivot.columns,
    y=heatmap_pivot.index,
    colorscale='Viridis',
    colorbar=dict(title='Unique Vessels')
))
fig_heatmap.update_layout(title='Unique Vessel Visits by Month and Ship Type', xaxis_title='Ship Type', yaxis_title='Month')

# --- Dashboard layout ---
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    html.H2("Bergen Port Vessel KPI Dashboard"),
    html.Div([
        html.Div([
            html.H5(f"Total unique vessels: {n_vessels}"),
            html.H5(f"Vessels that entered terminal area: {n_vessels_terminal}"),
        ], style={'margin-bottom': '20px'}),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_monthly), md=6),
            dbc.Col(dcc.Graph(figure=fig_visits_hist), md=6),
        ]),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_visits_pie), md=6),
            dbc.Col(dcc.Graph(figure=fig_scatter), md=6),
        ]),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_box_visit), md=6),
            dbc.Col(dcc.Graph(figure=fig_box_term), md=6),
        ]),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_stops_hist), md=6),
            dbc.Col(dcc.Graph(figure=fig_heatmap), md=6),
        ]),
        html.H4("KPI Table (first 20 rows):"),
        dash_table.DataTable(
            data=summary.head(20).to_dict('records'),
            columns=[{"name": i, "id": i} for i in summary.columns],
            page_size=20,
            style_table={'overflowX': 'auto', 'margin-top': '20px'},
            style_cell={'textAlign': 'left', 'padding':'3px'},
            style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'}
        ),
    ])
], fluid=True)

# --- Run dashboard in notebook (no more JupyterDash needed) ---
# "inline" mode for Jupyter, "external" opens new tab; choose your favorite
app.run(debug=True, mode='inline')


In [15]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from dash import Dash, dcc, html, dash_table, Input, Output, State
import dash_bootstrap_components as dbc

# Load summary and add month
summary = pd.read_csv("Bergen_kpi_summary.csv", parse_dates=['visit_start_time', 'visit_end_time'])
vessels_to_exclude = summary.loc[summary['stops_count'] > 5, 'mmsi'].unique()
summary = summary[~summary['mmsi'].isin(vessels_to_exclude)].reset_index(drop=True)
summary['month'] = summary['visit_start_time'].dt.strftime('%Y-%m')

# Unique vessels per month in port (bar)
monthly_counts = summary.groupby('month')['mmsi'].nunique().reset_index(name='unique_vessels')

# Unique vessels per month in terminal (bar)
summary['visited_terminal'] = summary['terminal_duration_hr'] > 0
terminal_visits_month = summary[summary['visited_terminal']].groupby('month')['mmsi'].nunique().reset_index(name='unique_terminal_vessels')

# Visits per vessel
visits_per_vessel = summary.groupby('mmsi')['visit_num'].max().reset_index(name='total_visits')
visits_hist = visits_per_vessel['total_visits'].value_counts().sort_index()

# Stops per visit
stops_per_visit = summary['stops_count']

# Box and scatter plots for turnaround time at terminal vs vessel length (core KPI)
terminal_visits = summary[summary['terminal_duration_hr'] > 0].copy()
terminal_visits['length_bin'] = pd.cut(terminal_visits['length'], bins=[0, 100, 150, 200, 400], labels=["0-100m", "100-150m", "150-200m", "200m+"])

# Colors (custom palette)
bar_color = "#3A6351"
terminal_color = "#489FB5"
hist_color = "#F7B801"
scatter_color = "#005F73"

# App
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    html.H2("Bergen Port Vessel KPI Dashboard (Curated Visuals)", style={'color': '#22223b', 'marginBottom': 20}),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id="bar-unique-vessels-port",
                figure=px.bar(
                    monthly_counts, x='month', y='unique_vessels',
                    title="Unique Vessels Per Month (Port Area)",
                    labels={"month": "Month", "unique_vessels": "Unique Vessels"},
                    color_discrete_sequence=[bar_color]
                ).update_layout(plot_bgcolor='white', yaxis=dict(gridcolor='gainsboro'))
            )
        ], md=6),
        dbc.Col([
            dcc.Graph(
                id="bar-unique-vessels-terminal",
                figure=px.bar(
                    terminal_visits_month, x='month', y='unique_terminal_vessels',
                    title="Unique Vessels Per Month (Terminal Area)",
                    labels={"month": "Month", "unique_terminal_vessels": "Unique Vessels (Terminal)"},
                    color_discrete_sequence=[terminal_color]
                ).update_layout(plot_bgcolor='white', yaxis=dict(gridcolor='gainsboro'))
            )
        ], md=6),
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id="bar-visits-per-vessel",
                figure=px.bar(
                    x=visits_hist.index, y=visits_hist.values,
                    title="Number of Port Visits per Vessel<br><span style='font-size:0.8em;color:gray'>Click any bar to see MMSIs</span>",
                    labels={"x": "Number of Visits", "y": "Number of Vessels"},
                    color_discrete_sequence=[terminal_color]
                ).update_traces(marker_line_color='white', marker_line_width=1.5)
                 .update_layout(plot_bgcolor='white', yaxis=dict(gridcolor='gainsboro'))
            ),
            html.Div(id='bar-click-mmsi-table')
        ], md=12)
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id="hist-stops-per-visit",
                figure=px.histogram(
                    summary, x='stops_count', nbins=summary['stops_count'].max()+1,
                    title="Stops Per Visit Distribution",
                    labels={"stops_count": "Stops per Visit", "count": "Visit Count"},
                    color_discrete_sequence=[hist_color]
                ).update_layout(plot_bgcolor='white', yaxis=dict(gridcolor='gainsboro'))
            ),
        ], md=6),
        dbc.Col([
            dcc.Graph(
                id="box-turnaround-vs-length",
                figure=px.box(
                    terminal_visits, x="length_bin", y="terminal_duration_hr",
                    title="Turnaround Time at Terminal vs. Vessel Length",
                    labels={"terminal_duration_hr": "Turnaround Time at Terminal (hrs)", "length_bin": "Vessel Length Bin"},
                    color_discrete_sequence=[scatter_color]
                ).update_layout(plot_bgcolor='white', yaxis=dict(gridcolor='gainsboro'))
            ),
        ], md=6),
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id="scatter-turnaround-vs-length",
                figure=px.scatter(
                    terminal_visits, x="length", y="terminal_duration_hr",
                    title="Turnaround Time at Terminal (Scatter: Length vs. Time)",
                    labels={"length": "Vessel Length (m)", "terminal_duration_hr": "Turnaround Time at Terminal (hrs)"},
                    color_discrete_sequence=[bar_color]
                ).update_traces(marker=dict(size=9, opacity=0.6))
                 .update_layout(plot_bgcolor='white', yaxis=dict(gridcolor='gainsboro'))
            ),
        ], md=12)
    ]),
    html.H4("KPI Table (filterable, first 20 rows):", style={'marginTop': 30}),
    dash_table.DataTable(
        data=summary.head(20).to_dict('records'),
        columns=[{"name": i, "id": i} for i in summary.columns],
        page_size=20,
        filter_action="native",
        sort_action="native",
        style_table={'overflowX': 'auto', 'margin-top': '20px'},
        style_cell={'textAlign': 'left', 'padding':'4px', 'fontSize': 13},
        style_header={'backgroundColor': '#E8E8E8', 'fontWeight': 'bold'}
    ),
], fluid=True)

# --- Callback: Show MMSIs for clicked bar on "visits per vessel" graph ---
@app.callback(
    Output('bar-click-mmsi-table', 'children'),
    Input('bar-visits-per-vessel', 'clickData')
)
def display_mmsis(clickData):
    if clickData:
        num_visits = clickData['points'][0]['x']
        vessels_list = visits_per_vessel.loc[visits_per_vessel['total_visits'] == num_visits, 'mmsi'].tolist()
        if vessels_list:
            return html.Div([
                html.H6(f"Vessels with {num_visits} visits:"),
                dash_table.DataTable(
                    data=[{'mmsi': m} for m in vessels_list],
                    columns=[{"name": "mmsi", "id": "mmsi"}],
                    page_size=10,
                    style_table={'maxHeight': '180px', 'overflowY': 'auto'},
                    style_cell={'fontSize': 12}
                )
            ], style={'marginTop': '15px'})
    return ""

app.run(debug=True, mode='inline')
